# Standardizing the Published Model and Comparing to the ModelSEED Model

In [ ]:
%run util.py
import cobra
print(cobra.__version__)
pubmod = MSModelUtil.from_cobrapy("data/PublishedModel.XML")
util.load_kbase_gene_container("Acinetobacter_baylyi_ADP1_RPG_Snekmer_LA_Pfam",ws=219173,localname="ADP1")
translation = {}
for gene in pubmod.model.genes:
    ftrs = util.alias_to_ftrs("ADP1",gene.id)
    if len(ftrs) > 0:
        translation[gene.id] = ftrs[0]
util.save("gene_translation",translation)
from cobra.manipulation.modify import rename_genes
rename_genes(pubmod.model, translation)
msmodel = util.get_model("179225/Abaylyi_ADP1_RASTMS2_OMEGGA_Abaylyi_ADP1_RAST.mdlMS2_OMEGGA_iAbaylyi_Carbon_Succinic.gf")
gene_remove_list = []
for gene in msmodel.model.genes:
    if gene.id[0:4] == "mRNA":
        gene_remove_list.append(gene)
for gene in gene_remove_list:
    msmodel.model.genes.remove(gene)
output = util.model_standardization(pubmod, msmodel=msmodel,filter_based_on_template=False)
util.save("match_stats",output["match_stats"])
util.save("model_match_stats",output["model_match_stats"])
util.display_dataframe(df=output["rxndf"])

# Using AI to review all model reactions and flag errors in directionality, definition, or stoichiometry

In [1]:
%run util.py
model = util.get_model("179225/Abaylyi_ADP1_RASTMS2_OMEGGA_Abaylyi_ADP1_RAST.mdlMS2_OMEGGA_iAbaylyi_Carbon_Succinic.gf")
output = {}
for rxn in model.model.reactions:
    output[rxn.id] = util._parse_rxn_stoichiometry(rxn)
util.save("mdl_rxn_hash",output)
#output = util.ai_analysis_of_model_reactions(model)
#til.display_dataframe(df=output)

modelseedpy 0.4.2


2025-08-28 11:27:38,074 - __main__.NotebookUtil - INFO - Notebook environment detected
2025-08-28 11:27:38,075 - __main__.NotebookUtil - INFO - Loaded 0 tokens from /Users/chenry/.tokens
2025-08-28 11:27:38,075 - __main__.NotebookUtil - INFO - Loaded kbase tokens from /Users/chenry/.kbase/token
2025-08-28 11:27:38,076 - __main__.NotebookUtil - INFO - ArgoGatewayClient initialised | model=gpto3mini env=dev timeout=120.0s url=https://apps-dev.inside.anl.gov/argoapi/api/v1/resource/streamchat/
2025-08-28 11:27:38,090 - __main__.NotebookUtil - WARNING - Section 'ModelSEEDBiochem' not found in config


cobrakbase 0.4.0


2025-08-28 11:27:42,755 - __main__.NotebookUtil - INFO - ModelSEED database loaded from /Users/chenry/acode/KBUtilLib/src/kbutillib/dependencies/ModelSEEDDatabase


In [2]:
value_counts = output["combined"].value_counts()
print(value_counts)

combined
>|>|>       311
=|=|=       288
>|>|=       280
=|>|=       206
<|<|<        28
<|<|=        26
>|=|=        24
=|<|=        19
=|>|>        18
=|?|=        12
<|>|=        12
<|=|=        10
>|=|>         8
=|=|>         7
=|>|<         6
=|=|<         4
=|<|<         4
<|?|=         3
<|>|<         2
>|?|=         1
<|=|<         1
<|?|<         1
>|<|=         1
<|>|>         1
>|=|<         1
>|<|>         1
>|?|>         1
>|>|<         1
=|=|None      1
Name: count, dtype: int64


# Evaluating impact of directionality changes on flux solutions

In [1]:
%run util.py
from cobra.flux_analysis import pfba, moma
import cobra
import pandas as pd
import numpy as np

# Load the model
model = util.get_model("179225/Abaylyi_ADP1_RASTMS2_OMEGGA_Abaylyi_ADP1_RAST.mdlMS2_OMEGGA_iAbaylyi_Carbon_Succinic.gf")

# Add DGOA reaction
util.add_dgoa_reaction(model)

# Set up media
pyruvate_media = util.get_media("KBaseMedia/Carbon-Pyruvic-Acid")
model.set_media(pyruvate_media)
model.model.objective = "bio1"

# 1. Wild type analysis - rxn01332 active, DGOA inactive
print("Running wild type analysis...")
model.model.reactions.get_by_id("rxn01332_c0").lower_bound = -1000
model.model.reactions.get_by_id("rxn01332_c0").upper_bound = 1000
model.model.reactions.get_by_id("DgoA").lower_bound = 0
model.model.reactions.get_by_id("DgoA").upper_bound = 0

output = util.ai_analysis_of_model_reactions(model,output_dataframe=False)

records = []
wt_solution = pfba(model.model)
for rxn in model.model.reactions:
    if rxn.id not in output:
        print("Reaction not found in output:", rxn.id)
    else:
        if wt_solution.fluxes[rxn.id] > 0.000001:
            if output[rxn.id]["ai_direction"] == "<":
                original = [model.model.reactions.get_by_id(rxn.id).lower_bound,model.model.reactions.get_by_id(rxn.id).upper_bound]
                model.model.reactions.get_by_id(rxn.id).lower_bound = -1000
                model.model.reactions.get_by_id(rxn.id).upper_bound = 0
                new_solution = pfba(model.model)
                status = "fail - nogrowth"
                if new_solution.status == "optimal" and new_solution.fluxes["bio1"] > 0.01:
                    status = "success"
                else:
                    status = "fail - "+new_solution.status
                    model.model.reactions.get_by_id(rxn.id).lower_bound = original[0]
                    model.model.reactions.get_by_id(rxn.id).upper_bound = original[1]
                records.append({
                    "reaction": rxn.id,
                    "name": rxn.name,
                    "equation": output[rxn.id]["equation"],
                    "conflict":"><",
                    "flux": wt_solution.fluxes[rxn.id],
                    "ai_direction": output[rxn.id]["ai_direction"],
                    "model_direction": output[rxn.id]["model_direction"],
                    "biochem_reversibility": output[rxn.id]["biochem_reversibility"],
                    "status": status
                })
        elif wt_solution.fluxes[rxn.id] < -0.000001:
            if output[rxn.id]["ai_direction"] == ">":
                original = [model.model.reactions.get_by_id(rxn.id).lower_bound,model.model.reactions.get_by_id(rxn.id).upper_bound]
                model.model.reactions.get_by_id(rxn.id).lower_bound = 0
                model.model.reactions.get_by_id(rxn.id).upper_bound = 1000
                new_solution = pfba(model.model)
                status = "fail - nogrowth"
                if new_solution.status == "optimal" and new_solution.fluxes["bio1"] > 0.01:
                    status = "success"
                else:
                    status = "fail - "+new_solution.status
                    model.model.reactions.get_by_id(rxn.id).lower_bound = original[0]
                    model.model.reactions.get_by_id(rxn.id).upper_bound = original[1]
                records.append({
                    "reaction": rxn.id,
                    "name": rxn.name,
                    "equation": output[rxn.id]["equation"],
                    "conflict":"<>",
                    "flux": wt_solution.fluxes[rxn.id],
                    "ai_direction": output[rxn.id]["ai_direction"],
                    "model_direction": output[rxn.id]["model_direction"],
                    "biochem_reversibility": output[rxn.id]["biochem_reversibility"],
                    "status": status
                })
records = pd.DataFrame.from_records(records)
util.display_dataframe(df=records)


modelseedpy 0.4.2


2025-08-26 00:18:18,445 - __main__.NotebookUtil - INFO - Notebook environment detected
2025-08-26 00:18:18,445 - __main__.NotebookUtil - INFO - Loaded 0 tokens from /Users/chenry/.tokens
2025-08-26 00:18:18,446 - __main__.NotebookUtil - INFO - Loaded kbase tokens from /Users/chenry/.kbase/token
2025-08-26 00:18:18,447 - __main__.NotebookUtil - INFO - ArgoGatewayClient initialised | model=gpto3mini env=dev timeout=120.0s url=https://apps-dev.inside.anl.gov/argoapi/api/v1/resource/streamchat/
2025-08-26 00:18:18,461 - __main__.NotebookUtil - WARNING - Section 'ModelSEEDBiochem' not found in config


cobrakbase 0.4.0


2025-08-26 00:18:22,947 - __main__.NotebookUtil - INFO - ModelSEED database loaded from /Users/chenry/acode/KBUtilLib/src/kbutillib/dependencies/ModelSEEDDatabase


Running wild type analysis...
Reaction not found in output: bio1
Reaction not found in output: EX_cpd00246_e0
Reaction not found in output: EX_cpd00065_e0
Reaction not found in output: EX_cpd00122_e0
Reaction not found in output: EX_cpd00268_e0
Reaction not found in output: EX_cpd00136_e0
Reaction not found in output: EX_cpd00363_e0
Reaction not found in output: EX_cpd00182_e0
Reaction not found in output: EX_cpd00053_e0
Reaction not found in output: EX_cpd11576_e0
Reaction not found in output: EX_cpd00412_e0
Reaction not found in output: EX_cpd17027_e0
Reaction not found in output: EX_cpd00367_e0
Reaction not found in output: EX_cpd11575_e0
Reaction not found in output: EX_cpd00054_e0
Reaction not found in output: EX_cpd00418_e0
Reaction not found in output: EX_cpd00041_e0
Reaction not found in output: EX_cpd03696_e0
Reaction not found in output: EX_cpd00179_e0
Reaction not found in output: EX_cpd00060_e0
Reaction not found in output: EX_cpd11596_e0
Reaction not found in output: EX_cp

/opt/anaconda3/envs/modelseed/lib/python3.9/site-packages/itables/typing.py:201: SyntaxWarning: These arguments are not documented in ITableOptions: {'searching', 'info'}. You can silence this warning by setting `itables.options.warn_on_undocumented_option=False`. If you believe ITableOptions should be updated, please make a PR or open an issue at https://github.com/mwouts/itables
  warnings.warn(
/opt/anaconda3/envs/modelseed/lib/python3.9/site-packages/itables/typing.py:201: SyntaxWarning: These arguments are not documented in DTForITablesOptions: {'searching', 'info'}. You can silence this warning by setting `itables.options.warn_on_undocumented_option=False`. If you believe ITableOptions should be updated, please make a PR or open an issue at https://github.com/mwouts/itables
  warnings.warn(


Loading ITables v2.4.4 from the internet... (need help?)


None